In [1]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(stringr))
library(reshape2)
library(data.table)
library(parallel)
library(purrr)
source('/home//jupyter/BRI_Figures_Final_V2/helper_function/helper_function_IHA.r')


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths



Attaching package: ‘data.table’


The following objects are masked from ‘package:reshape2’:

    dcast, melt


The following objects are masked from ‘package:dplyr’:

    between, first, last



Attaching package: ‘purrr’


The following object is masked from ‘package:data.table’:

    transpose




# Read Meta Data

In [2]:
meta_data_selected_RNA <- read.csv('/home/jupyter/BRI_Figures_Final_V1/Dataset/scRNA_meta_data-2024-05-09.csv')

meta_data_selected_RNA$sample.drawDate <- as.Date(meta_data_selected_RNA$sample.drawDate)

meta_data_selected_RNA <- meta_data_selected_RNA %>%
  mutate(Flu_Year = case_when(
    sample.drawDate <= as.Date('2020-07-01') ~ '2019-2020',
    sample.drawDate > as.Date('2020-07-01') & sample.drawDate <= as.Date('2021-07-01') ~ '2020-2021',
    sample.drawDate > as.Date('2021-07-01') & sample.drawDate <= as.Date('2022-07-01') ~ '2021-2022',
    sample.drawDate > as.Date('2022-07-01') & sample.drawDate <= as.Date('2023-07-01') ~ '2022-2023',
    TRUE ~ NA_character_
  ))

meta_data_selected_RNA <- meta_data_selected_RNA %>%
  mutate(Flu_Day = case_when(
    sample.visitName	%in% c("Flu Year 1 Day 0","Flu Year 2 Day 0") ~ 'Day 0',
    sample.visitName	%in% c("Flu Year 1 Day 7","Flu Year 2 Day 7") ~ 'Day 7',
    sample.visitName	%in% c("Flu Year 1 Day 90","Flu Year 2 Day 90") ~ 'Day 90',
    TRUE ~ sample.visitName
  ))

In [3]:
meta_data_selected_RNA_D0_D7<-meta_data_selected_RNA %>% filter(Flu_Year%in% c("2021-2022","2020-2021")) %>% filter(Flu_Day %in% c("Day 0",'Day 7'))

In [4]:
selected_donor<-as.data.frame(table(meta_data_selected_RNA_D0_D7$subject.subjectGuid)) %>% filter(Freq==4)

In [5]:
meta_data_selected<-meta_data_selected_RNA_D0_D7 %>% filter(subject.subjectGuid %in% selected_donor$Var1)
write.csv(meta_data_selected,"selected_samples_with_acutal_flu_year.csv")

In [6]:
meta_data_selected_RNA_D0_D7<-meta_data_selected_RNA_D0_D7 %>% filter(subject.subjectGuid %in% selected_donor$Var1,Flu_Year%in% c("2020-2021"))

# Read Freq Files and do subsetting

In [7]:
df_freq<-read.csv("/home/jupyter/BRI_Figures_Final_V2/Dataset/diha_AIFI_L3_frequencies_2024-05-05.csv")

In [8]:
df_freq_Y1<-df_freq %>% filter(specimen.specimenGuid %in% meta_data_selected_RNA_D0_D7$pbmc_sample_id)

In [9]:
unique(df_freq_Y1$AIFI_L3)

[1] "CD27+ effector B cell"               
 [2] "CD27- effector B cell"               
 [3] "Activated memory B cell"             
 [4] "CD95 memory B cell"                  
 [5] "Core memory B cell"                  
 [6] "Early memory B cell"                 
 [7] "Type 2 polarized memory B cell"      
 [8] "Core naive B cell"                   
 [9] "ISG+ naive B cell"                   
[10] "Plasma cell"                         
[11] "Transitional B cell"                 
[12] "ASDC"                                
[13] "cDC1"                                
[14] "CD14+ cDC2"                          
[15] "HLA-DRhi cDC2"                       
[16] "ISG+ cDC2"                           
[17] "pDC"                                 
[18] "ILC"                                 
[19] "Core CD14 monocyte"                  
[20] "IL1B+ CD14 monocyte"                 
[21] "ISG+ CD14 monocyte"                  
[22] "C1Q+ CD16 monocyte"                  
[23] "Core CD16 monocyte"                  
[24] "ISG+ CD16 monocyte"                  
[25] "Intermediate monocyte"               
[26] "CD56bright NK cell"                  
[27] "Adaptive NK cell"                    
[28] "GZMK+ CD56dim NK cell"               
[29] "GZMK- CD56dim NK cell"               
[30] "ISG+ CD56dim NK cell"                
[31] "Proliferating NK cell"               
[32] "Platelet"                            
[33] "CLP cell"                            
[34] "CMP cell"                            
[35] "CD8aa"                               
[36] "DN T cell"                           
[37] "CD4 MAIT"                            
[38] "CD8 MAIT"                            
[39] "CM CD4 T cell"                       
[40] "GZMB- CD27+ EM CD4 T cell"           
[41] "GZMB- CD27- EM CD4 T cell"           
[42] "ISG+ memory CD4 T cell"              
[43] "KLRF1- GZMB+ CD27- memory CD4 T cell"
[44] "CM CD8 T cell"                       
[45] "GZMK+ CD27+ EM CD8 T cell"           
[46] "GZMK- CD27+ EM CD8 T cell"           
[47] "ISG+ memory CD8 T cell"              
[48] "KLRF1+ GZMB+ CD27- EM CD8 T cell"    
[49] "KLRF1- GZMB+ CD27- EM CD8 T cell"    
[50] "Core naive CD4 T cell"               
[51] "ISG+ naive CD4 T cell"               
[52] "SOX4+ naive CD4 T cell"              
[53] "Core naive CD8 T cell"               
[54] "SOX4+ naive CD8 T cell"              
[55] "Proliferating T cell"                
[56] "KLRB1+ memory CD4 Treg"              
[57] "KLRB1+ memory CD8 Treg"              
[58] "Memory CD4 Treg"                     
[59] "Memory CD8 Treg"                     
[60] "Naive CD4 Treg"                      
[61] "GZMB+ Vd2 gdT"                       
[62] "GZMK+ Vd2 gdT"                       
[63] "KLRF1- effector Vd1 gdT"             
[64] "Naive Vd1 gdT"                       
[65] "SOX4+ Vd1 gdT"                       
[66] "Erythrocyte"                         
[67] "ISG+ MAIT"                           
[68] "ISG+ naive CD8 T cell"               
[69] "GZMK+ memory CD4 Treg"               
[70] "BaEoMaP cell"                        
[71] "KLRF1+ effector Vd1 gdT"

In [10]:
df_freq_Y1<-df_freq_Y1 %>% filter(AIFI_L3 %in% c("CD27+ effector B cell", 
                                                 "CD95 memory B cell", 'Type 2 polarized memory B cell',
                    "CD27- effector B cell", 
                    "Core memory B cell", 
                    "Plasma cell"))

In [11]:
df_freq_Y1<-left_join(df_freq_Y1,meta_data_selected_RNA_D0_D7[c('pbmc_sample_id','Flu_Year','Flu_Day')],
                      by=c("specimen.specimenGuid"="pbmc_sample_id"))

In [12]:
write.csv(df_freq_Y1,"freq_actual_flu_year_1_D0_D7.csv")